# 書くこと


- [参考のノートブック](https://www.kaggle.com/artgor/segmentation-in-pytorch-using-convenient-tools)

# はじめに

本文書は，日本コンピュータ外科学会によって開催されたJSCAS AI Challengeという機械学習のコンペティションに参加したときの記録です．

https://www.jscas.org/business/2020/09/c83412be4feb3ca543c74d759fee400c073cda07.html


この文書では大きく分けて2つの話題，コンペティションまでに行ったこと，コンペティション後に他の人の発表を参考に改善を試みたこと，について書きます．

コンペティションの発表までに行ったこととしては，モデルの選定，損失関数の調査，パラメータチューニングなどがあります．

また，アンサンブル学習を行い，結果的にこれが最も効果のある学習方法でした．
今回は主にこのアンサンブル学習について実際にどのように行ったのか書きます．

コンペティション後には，他の人の発表を参考にしながら改善を試みました．
こちらについては結果的に大きな精度の向上は達成できませんでしたが，
いくつか実験を行ったので記録として書きます．


# コンペティション概要

先に述べた通り，参加したコンペティションは，日本コンピュータ外科学会によって開催されたJSCAS AI Challengeという名前の機械学習コンペティションです．

コンペティションで要求されるタスクは，内視鏡外科手術画像の中から術具の先端部分を自動で識別せよ，というセマンティックセグメンテーションタスクです．

具体的には超音波凝固切開装置と呼ばれる医療器具が画像に写っており，
切開を行う先端部分を認識することが求められていることです．

また，この超音波凝固切開装置には新旧の二種類が存在し，それを扱うか否かによって以下の2つのタスクレベルに分けられていました．

- レベル1：新旧の区別無く術具の先端を認識
- レベル2：新旧の区別（クラスタリング）を行いかつ，術具の先端を認識

今回私はレベル1のみを行いました（機械学習初心者でクラスタリングまで手が回らなかったためです）．

## 評価指標

評価指標にはF値を使います．
セマンティックセグメンテーションでは各ピクセルごとに正誤判定ができます．
正誤判定において正と判定されたもののうち実際に正であるものの割合をprecision,
実際に正であるもののうち正と判定されたものの割合をrecallと呼び，
precisionとrecallの調和平均をF値と呼びます．

## データセット

データセットは以下の通りです．

- トレーニング画像
    - 325枚
- テスト画像
    - 75枚
- 検証画像
    - 150枚（新：75枚，旧：75枚）
    - セグメンテーションの難易度毎にeasy, middle, hardに分類される
    
すべての画像には対応するラベル画像があり，
ラベル画像は背景(0)，旧(1)，新(2)が各ピクセルに割り当てられています．
    
ラベルを含む画像データはJSCASのページに公開されています．
http://bit.ly/ai20data
    

# 手法

書くこと
- どのようにデータの前処理をおこなったか
- どのような識別器を用いたか
    - モデル
    - 損失関数
    - 最適化アルゴリズム
- どのように術具の領域を推定したか


## 前処理

トレーニング画像325枚に対して，精度向上を目的として,いくつかの前処理を行う
- mixup
- Data Augumentation(DA)

### mixup

二枚の画像を割合で混ぜる  
$\lambda \times x_1 + (1 - \lambda) \times x_2$  
$\lambda \sim Beta(\alpha, \alpha)$
今回$\alpha$は0.4にした

### DataAugmentation(以下DA)

- 平行移動，反転などのアフィン変換
- 明るさ
- 色の濃度の変更

具体的なコードを貼る

## 学習モデル

- DeepLabV3+を使用
    - 参考実装のu-netより学習速度が早い
    
DeepLabv3+はGoogleが2018年に出したモデルで，空間ピラミッドプーリング
モジュールと、Encoder-Decoder モデルの利点を組み合わせたモデルです．
詳しくは論文やQiitaのまとめなどを参照．
https://qiita.com/mine820/items/14e7c556b358dbc4ee9a

また，セマンティックセグメンテーションの代表的なモデルU-netとDeepLabv3+で学習速度を比較するとDeepLabv3+のほうが早く，高速にイテレーションを回すことができます．

参考にしたコードからの変更点としては，ドロップアウト率を0.5に変更したこと，があります．

### 損失関数

損失関数については，いくつかの調査を行いました．
セマンティックセグメンテーションでは，認識対象の領域と背景との比率が大きく異る場合があります．
今回扱うデータにおいても，術具の先端という非常に小さい領域が認識対象です．
そのような場合には一般的なbinary cross entroyよりもセマンティックセグメンテーションに特化した損失関数を用いると，精度が向上する場合があります．

今回は以下のような損失関数を検討しました．

- binary crossentropy
- sigmoid focal loss
- focal tversky
- boundary loss

結果としてfocal tverskyを基本的に使用しました．
後述するアンサンブル学習の関係で，一部の識別器ではboundary loss関数をfocal tverskyと併用している場合もあります．

## アンサンブル学習

複数の弱識別器を組み合わせて学習・推論を行う機械学習の手法をアンサンブル学習といいます．

今回は複数の分類器の出力の平均を取ることでより良い分類（セグメンテーション）結果を得る，という形でアンサンブル学習の考え方を取り入れています．

図で説明する．

コンペティションに提出した際に使った識別器は以下の３つです．
- DA有り($y_1$)
- DA無し($y_2$)
- 損失関数の違い($y_3$)
    
$(y_1 + y_2 + y_3) / 3$

# 評価

実際の推定結果を比較して，アンサンブル学習の効果を確かめます．

In [8]:
# DA

# create model
model1 = DeepLabv3()
model2 = DeepLabv3()
model3 = DeepLabv3()

# training
model1.train()
model2.train()
model3.train()

# estimation
result1 = predict(model1)
result2 = predict(model2)
result3 = predict(model3)
result = (result1+result2+result3)/3.0

NameError: name 'DeepLabv3' is not defined

学習した識別器の性能を確かめるために，テストデータを用いて，識別器の推定精度を算出します．
テストデータは識別難易度ごとにeasy,middle,hardに分かれており，それぞれについて推定精度を出します．
3つの識別器それぞれの推定結果とアンサンブル学習の結果を図示しています．
(hardについては，認識することが困難であったため，図から除外しています．)

![result](./jscas_ai_challenge_2020_data/compare4.png)

- FT+DA（青色）    
- FT（橙色）       
- FT+BL（緑色）    
- ensenbled（赤色）

青色のグラフはfocal tversky + data augmentation,
橙色のグラフはfocal tversky,
緑色のグラフはfocal tversky + boundary loss,
赤色のグラフはアンサンブル学習の結果，
となっています．

アンサンブル学習に用いる識別器の選択は，今回はヒューリスティックに行いました．


アンサンブル学習によってすべての難易度で二番目の精度を達成していることがわかります．

それぞれの識別器に得意不得意なデータがありますが，アンサンブル学習を行うことによって，平均的に優秀な識別器を作成可能である，ということがわかりました．

# 評価の結果に基づいた改良

ここからはコンペティション発表後に行った実験について，いくつか紹介します．
結論から書くと識別性能に大きく寄与するような学習方法は発見できませんでしたが，記録として残しておこうと思います．
（特に識別できていなかったhardの画像を識別可能になることを期待したのですが，うまくいきませんでした）

さて，行ったこととしては，

- fog, DefocusBlurなどの画像処理をDAとして行う
- RGBチャンネルに加えて，RGBチャンネルに前処理を施したデータを4つ目以降の特徴量として扱う
- 上下反転のみでDAを行う

といったものがあります．




## fog, DefocusBlurなどの画像処理をDA

セグメンテーション精度が悪かった画像として，切開で焼き切るときに発生する煙で曇っていたり，カメラのピントがずれているものが多く見られました．
とりわけテストデータの識別難易度hardはそのような画像が多く，これが識別困難の原因である可能性があります．

そのような画像をDAで生成して識別器を訓練することで精度が向上するのではないだろうか，というのがこの試みです．

DAにはimgaugライブラリを用いました．
Fog(煙っぽい処理)やDefocusBlur(ピントのズレ)関数が用意されていて，簡単に画像を加工することができます．

具体例

In [4]:
import imgaug.augmenters as iaa

da_list = iaa.Sequential([
    iaa.imgcorruptlike.Fog(severity=2), # kemuri
    iaa.imgcorruptlike.DefocusBlur(severity=2), # camera no pinto
])

# 実際に画像を加工してみる
# 画像を出力するコード

`Sequential` は 順番に処理を適用します．
他にも渡した処理のうちいくつかを適用する`SomeOf`や，
一つだけ適用する`OneOf`など便利な関数があります．

結果

## データに合わせた特徴量の追加

内視鏡手術画像の特徴として内臓からなる赤色が多くを占めるというものがあります．
対して術具先端は黒に近い色で構成されているため，RGB空間でうまく分離できるのではないかと考えました．
RGBチャネルのみを入力特徴量として扱うことで，内臓と術具先端を分けることができるのが理想ですが，実際にはそうでないこともあるため，明示的に新しい特徴量を導入することでうまくいかないかという試みです．

今回は目的の処理を行うためにopencvのpythonライブラリを使いました．
`threshold`という関数があり閾値処理をして画像の二値化ができます．

In [7]:
import cv2

cv2.threshold(image_one_channel, 100, 255, cv2.THRESH_BINARY)

NameError: name 'image_one_channel' is not defined

渡す画像は0~255の値を持っています．
ピクセル値が第二引数で指定した値以上の場合，そのピクセル値を第三引数の値にします．
第四引数は数値の切り上げ方のオプションで，グラデーションで変化させることもできます．
詳しくはこのページを見てください．

## 上下反転のDA

実際にどのようなDAが効果的なのか，という疑問から試してみた一例です．


# まとめ

術具先端の識別を機械学習を用いて行った

効果的だった手法はアンサンブル手法

識別が困難な画像を模倣するようなDAをいくつか試したが，精度改善は見られなかった

# 参考文献

# tmp

In [ ]:


#　使用するモデル選択
model = Deeplabv3(weights=None, input_shape=(176, 320, 3), OS=8, classes=1, backbone='xception', activation='sigmoid', dropout_rate=0.5)

# モデル表示
import keras
from IPython.display import Image
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True)
Image(retina=True, filename='model.png')

